In [1]:
import ee
import geopandas as gpd
import geemap
import pandas as pd
import math  # Import the math module


def atm_corr(img):
    # print("Type of 'img' inside atm_corr function:", type(img))


    footprint = img.geometry()

    # DEM
    dem = ee.Image('USGS/SRTMGL1_003').clip(footprint)
    DEM_OLI = ee.Image(1);


    # ozone
    # DU_OLI = ee.Image(ozone.filterBounds(footprint).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month')).filter(ee.Filter.calendarRange(startYear, endYear, 'year')).mean())

    DU_OLI = ee.Image(300); # ozone @ sea level

    # Julian Day
    imgDate_OLI = ee.Date(img.get('system:time_start'))
    FOY_OLI = ee.Date.fromYMD(imgDate_OLI.get('year'),1,1)
    JD_OLI = imgDate_OLI.difference(FOY_OLI,'day').int().add(1)

    # Earth-Sun distance
    d_OLI = ee.Image.constant(img.get('EARTH_SUN_DISTANCE'))

# Sun elevation
    SunEl_OLI = ee.Image.constant(img.get('SUN_ELEVATION'))

    # Sun azimuth
    SunAz_OLI = img.select('SAA').multiply(ee.Image(0.01))

    # Satellite zenith
    SatZe_OLI = img.select('VZA').multiply(ee.Image(0.01))
    cosdSatZe_OLI = SatZe_OLI.multiply(ee.Image(math.pi).divide(180)).cos()
    sindSatZe_OLI = SatZe_OLI.multiply(ee.Image(math.pi).divide(180)).sin()

    

    # Satellite azimuth
    SatAz_OLI = img.select('VAA').multiply(ee.Image(0.01))

    # Sun zenith
    SunZe_OLI = img.select('SZA').multiply(ee.Image(0.01))
    cosdSunZe_OLI = SunZe_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image.constant(180))).cos() # in degrees
    sindSunZe_OLI = SunZe_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image.constant(180))).sin() # in degrees

 # Relative azimuth
    RelAz_OLI = ee.Image(SunAz_OLI)
    cosdRelAz_OLI = RelAz_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))).cos()
    

    # Pressure calculation
    P_OLI = ee.Image(101325).multiply(ee.Image(1).subtract(ee.Image(0.0000225577).multiply(DEM_OLI)).pow(5.25588)).multiply(0.01)
    Po_OLI = ee.Image(1013.25)
 # Radiometric Calibration
    # define bands to be converted to radiance
    bands_OLI = ['B1','B2','B3','B4','B5','B6','B7']

    # radiance_mult_bands
    rad_mult_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_MULT_BAND_1')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_2')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_3')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_4')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_5')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_6')),
                        ee.Image(img.get('RADIANCE_MULT_BAND_7'))])).toArray(1)

    # radiance add band
    rad_add_OLI = ee.Image(ee.Array([ee.Image(img.get('RADIANCE_ADD_BAND_1')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_2')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_3')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_4')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_5')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_6')),
                        ee.Image(img.get('RADIANCE_ADD_BAND_7'))]
                        )).toArray(1)

    # create an empty image to save new radiance bands to
    imgArr_OLI = img.select(bands_OLI).toArray().toArray(1);
    Ltoa_OLI = imgArr_OLI.multiply(rad_mult_OLI).add(rad_add_OLI)

 # print(Ltoa_OLI)
     # esun (extra-terrestrial solar irradiance) Units = mW cm-2 um-1
    ESUN_OLI = ee.Image.constant(197.24790954589844).\
      addBands(ee.Image.constant(201.98426818847656)).\
      addBands(ee.Image.constant(186.12677001953125)).\
      addBands(ee.Image.constant(156.95257568359375)).\
      addBands(ee.Image.constant(96.04714965820312)).\
      addBands(ee.Image.constant(23.8833221450863)).\
      addBands(ee.Image.constant(8.04995873449635)).toArray().toArray(1)
    ESUN_OLI = ESUN_OLI.multiply(ee.Image(1))

    ESUNImg_OLI = ESUN_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Ozone Correction
    # Ozone coefficients https://www.arm.gov/publications/tech_reports/doe-sc-arm-tr-129.pdf?id=811 (Appendix A) by band center (lambda)
    koz_OLI = ee.Image.constant(0.0039).addBands(ee.Image.constant(0.0218)).\
      addBands(ee.Image.constant(0.1078)).addBands(ee.Image.constant(0.0608)).addBands(ee.Image.constant(0.0019)).addBands(ee.Image.constant(0)).addBands(ee.Image.constant(0)).toArray().toArray(1)

    # Calculate ozone optical thickness
    Toz_OLI = koz_OLI.multiply(DU_OLI).divide(ee.Image.constant(1000));
    # Calculate TOA radiance in the absense of ozone
    Lt_OLI = Ltoa_OLI.multiply(((Toz_OLI)).multiply((ee.Image.constant(1).divide(cosdSunZe_OLI)).add(ee.Image.constant(1).divide(cosdSatZe_OLI))).exp())

    # Rayleigh optical thickness

    bandCenter_OLI = ee.Image(443).divide(1000).addBands(ee.Image(483).divide(1000)).addBands(ee.Image(561).divide(1000)).addBands(ee.Image(655).divide(1000)).addBands(ee.Image(865).divide(1000)).addBands(ee.Image(1609).divide(1000)).addBands(ee.Number(2201).divide(1000)).toArray().toArray(1)

     # create an empty image to save new Tr values to
    Tr_OLI = (P_OLI.divide(Po_OLI)).multiply(ee.Image(0.008569).multiply(bandCenter_OLI.pow(-4))).multiply((ee.Image(1).add(ee.Image(0.0113).multiply(bandCenter_OLI.pow(-2))).add(ee.Image(0.00013).multiply(bandCenter_OLI.pow(-4)))))

    # Fresnel Reflection
    # Specular reflection (s- and p- polarization states)
    theta_V_OLI = ee.Image(0.0000000001)
    sin_theta_j_OLI = sindSunZe_OLI.divide(ee.Image(1.333))

    theta_j_OLI = sin_theta_j_OLI.asin().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    theta_SZ_OLI = SunZe_OLI

    R_theta_SZ_s_OLI = (((theta_SZ_OLI.multiply(ee.Image.constant(math.pi)
                                                .divide(ee.Image(180)))).subtract(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))))).sin().pow(2)).divide((((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180)))).add(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(ee.Image(180))))).sin().pow(2)))

    R_theta_V_s_OLI = ee.Image(0.0000000001)

    R_theta_SZ_p_OLI = (((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(180))).subtract(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(180)))).tan().pow(2)).divide((((theta_SZ_OLI.multiply(ee.Image.constant(math.pi).divide(180))).add(theta_j_OLI.multiply(ee.Image.constant(math.pi).divide(180)))).tan().pow(2)))

    R_theta_V_p_OLI = ee.Image(0.0000000001)

    R_theta_SZ_OLI = ee.Image(0.5).multiply(R_theta_SZ_s_OLI.add(R_theta_SZ_p_OLI))

    R_theta_V_OLI = ee.Image(0.5).multiply(R_theta_V_s_OLI.add(R_theta_V_p_OLI))

 # Rayleigh scattering phase function
    # Sun-sensor geometry

    theta_neg_OLI = ((cosdSunZe_OLI.multiply(ee.Image(-1))).multiply(cosdSatZe_OLI)).subtract((sindSunZe_OLI).multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_neg_inv_OLI = theta_neg_OLI.acos().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    theta_pos_OLI = (cosdSunZe_OLI.multiply(cosdSatZe_OLI)).subtract(sindSunZe_OLI.multiply(sindSatZe_OLI).multiply(cosdRelAz_OLI))

    theta_pos_inv_OLI = theta_pos_OLI.acos().multiply(ee.Image(180).divide(ee.Image.constant(math.pi)))

    cosd_tni_OLI = theta_neg_inv_OLI.multiply((ee.Image.constant(math.pi)).divide(180)).cos() # in degrees

    cosd_tpi_OLI = theta_pos_inv_OLI.multiply((ee.Image.constant(math.pi)).divide(180)).cos() # in degrees

    Pr_neg_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tni_OLI.pow(2))))

    Pr_pos_OLI = ee.Image(0.75).multiply((ee.Image(1).add(cosd_tpi_OLI.pow(2))))

    # Rayleigh scattering phase function
    Pr_OLI = Pr_neg_OLI.add((R_theta_SZ_OLI.add(R_theta_V_OLI)).multiply(Pr_pos_OLI));

    # Calulate Lr,
    denom_OLI = ee.Image(4).multiply(ee.Image.constant(math.pi)).multiply(cosdSatZe_OLI)
    Lr_OLI = (ESUN_OLI.multiply(Tr_OLI)).multiply(Pr_OLI.divide(denom_OLI))

 # Rayleigh corrected radiance
    Lrc_OLI = (Lt_OLI.divide(ee.Image(10))).subtract(Lr_OLI)
    LrcImg_OLI = Lrc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rayleigh corrected reflectance
    prc_OLI = Lrc_OLI.multiply(ee.Image.constant(math.pi)).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI))
    prcImg_OLI = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])
    rhorc = prc_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Aerosol Correction
    # Bands in nm
    bands_nm_OLI = ee.Image(443).addBands(ee.Image(483)).addBands(ee.Image(561)).addBands(ee.Image(655)).addBands(ee.Image(865)).addBands(ee.Image(0)).addBands(ee.Image(0)).toArray().toArray(1)

    # Lam in SWIR bands
    Lam_6_OLI = LrcImg_OLI.select('B6')
    Lam_7_OLI = LrcImg_OLI.select('B7')

    # Calculate aerosol type
    eps_OLI = (((((Lam_7_OLI).divide(ESUNImg_OLI.select('B7'))).log()).subtract(((Lam_6_OLI).divide(ESUNImg_OLI.select('B6'))).log())).divide(ee.Image(2201).subtract(ee.Image(1609)))) #.multiply(water_mask)

    # Calculate multiple scattering of aerosols for each band
    Lam_OLI = (Lam_7_OLI).multiply(((ESUN_OLI).divide(ESUNImg_OLI.select('B7')))).multiply((eps_OLI.multiply(ee.Image(-1))).multiply((bands_nm_OLI.divide(ee.Image(2201)))).exp())

    # diffuse transmittance trans_OLI = Tr_OLI.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe_OLI)).exp()
    trans_OLI = Tr_OLI.multiply(ee.Image(-1)).divide(ee.Image(2)).multiply(ee.Image(1).divide(cosdSatZe_OLI)).exp()


    # Compute water-leaving radiance
    Lw_OLI = Lrc_OLI.subtract(Lam_OLI).divide(trans_OLI)

    # water-leaving reflectance
    pw_OLI = (Lw_OLI.multiply(ee.Image.constant(math.pi)).multiply(d_OLI.pow(2)).divide(ESUN_OLI.multiply(cosdSunZe_OLI)))
    pwImg_OLI = pw_OLI.arrayProject([0]).arrayFlatten([bands_OLI])

    # Rrs
    Rrs = (pw_OLI.divide(ee.Image.constant(math.pi)).arrayProject([0]).arrayFlatten([bands_OLI]).slice(0,5))
    # .multiply(mask)
    Rrs = Rrs.updateMask(Rrs.gt(0));
    
    processed_image = ee.Image(img).addBands(Rrs)
    
    # Optionally, you can set metadata for the processed image if needed
    processed_image = processed_image.set('atmospheric_correction_applied', True)
    
    return processed_image
    # return Rrs.set('system:time_start',img.get('system:time_start'))


import ee
import geopandas as gpd
import geemap
import pandas as pd
import math  # Import the math module


# Initialize Earth Engine
ee.Initialize()

# Read the shapefile into a GeoDataFrame
wbd = gpd.read_file('HU12-ADK.shp')

# Define the Landsat 5 bands and their corresponding standard names
# 'B1','B2','B3','B4','B5','B6','B7'

LC5_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7','SZA', 'VZA', 'VAA','SAA']
L8_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B10', 'B7','SZA', 'VZA', 'VAA','SAA']
STD_NAMES = ['B1','B2','B3','B4','B5','B6','B7', 'SZA', 'VZA', 'VAA','SAA']



wbd = gpd.read_file('HU12-ADK.shp')







#Define a function to compute the mean and standard deviation of reflectance values for each band
def reflectance(img, lake_point):
    # Define the reducers for mean and standard deviation
    reducer_mean = ee.Reducer.mean()
    reducer_stdDev = ee.Reducer.stdDev()
    
    # Reduce the image to compute mean and standard deviation for each band
    reflectance_values = img.reduceRegion(
        reducer=reducer_mean.combine(reducer2=reducer_stdDev, sharedInputs=True), 
        geometry=lake_point, 
        scale=60
    )
    
    return img.set('DATE_SMP', img.date().format()).set('reflectance', reflectance_values)




# Initialize an empty list to store the filtered DataFrames for each lake
filtered_dfs = []

# Define Landsat paths and rows of interest
paths = [14]
rows = [29, 30]

# Merge paths and rows into a single list of tuples
path_row_combinations = [(path, row) for path in paths for row in rows]


# Loop over each lake in the GeoDataFrame
# Loop over each lake in the GeoDataFrame
for path, row in path_row_combinations:
        
    # Filter Landsat 5 imagery for the specific path and row
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1') \
        .filter(ee.Filter.eq('WRS_PATH', path)) \
        .filter(ee.Filter.eq('WRS_ROW', row)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .select(LC5_BANDS, STD_NAMES) \
        .map(atm_corr)



    # Filter Landsat 8 imagery for the specific path and row
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
        .filter(ee.Filter.eq('WRS_PATH', path)) \
        .filter(ee.Filter.eq('WRS_ROW', row)) \
        .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .select(L8_BANDS, STD_NAMES) \
        .map(atm_corr)

    

    # Merge the Landsat 5 and Landsat 8 collections
    l = l5.merge(l8)


    for index, lake_row in wbd.iterrows():  # Rename 'row' variable to 'lake_row'
        # Retrieve lake name and geometry
        lake_name = lake_row['name']
        lake_point = lake_row['geometry']
        huc12  = lake_row['huc12']                                                                                         
    
        specific_lake = wbd[wbd['name'] == lake_name]
    
        # Extract the geometry of the specific lake
        specific_lake_geometry = specific_lake.geometry.values[0]
        
        # Convert the geometry to GeoJSON format
        specific_lake_geojson = specific_lake_geometry.__geo_interface__
        
        map_reflectance = l.map(lambda img: reflectance(img, specific_lake_geojson))

        # Reduce the mapped image collection to get reflectance values for the specific path and row
        list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)

        # Convert the results to a pandas DataFrame
        df_reflectance = pd.DataFrame(list_reflectance.getInfo(), columns=['DATE_SMP', 'reflectance'])
    
        df_reflectance['DATE_SMP'] = pd.to_datetime(df_reflectance['DATE_SMP'])
        df_reflectance['DATE_SMP'] = df_reflectance['DATE_SMP'].dt.date
        df_reflectance['reflectance'] = df_reflectance['reflectance'].apply(
            lambda x: {k: v for k, v in x.items() if v is not None})
    
        # Unpack the 'reflectance' dictionary and create separate columns for each band
        df_reflectance = pd.concat([df_reflectance.drop('reflectance', axis=1),
                                    df_reflectance['reflectance'].apply(pd.Series).astype('float64', errors='ignore')], axis=1)
    
        # Add a new column for the lake name
        df_reflectance['PONDNAME'] = lake_name
        df_reflectance['WRS_PATH'] = path
        df_reflectance['WRS_ROW'] = row  # Use the 'row' variable from the outer loop
        df_reflectance['huc12'] = huc12

    
        # Add the DataFrame to the list
        filtered_dfs.append(df_reflectance)

# Concatenate all filtered DataFrames into a single DataFrame
df_filtered_lakes = pd.concat(filtered_dfs, ignore_index=True)

# Sort the DataFrame by 'DATE' in ascending order
df_filtered_lakes.sort_values(by='DATE_SMP', inplace=True)

# Display the DataFrame
df_filtered_lakes

,DATE_SMP,PONDNAME,WRS_PATH,WRS_ROW,huc12,B1_1_mean,B1_1_stdDev,B1_mean,B1_stdDev,B2_1_mean,...,SAA_mean,SAA_stdDev,SZA_mean,SZA_stdDev,VAA_mean,VAA_stdDev,VZA_mean,VZA_stdDev,B5_1_mean,B5_1_stdDev
0,1984-04-14,Upper Kayaderosseras Creek,14,29,020200030401,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18676,1984-04-14,Valentine Pond-Schroon River,14,29,020200010605,NaN,NaN,NaN,NaN,NaN,...,13697.214060,4.099635,4151.040148,1.609418,12421.025175,7146.644851,35.534669,13.941511,NaN,NaN
61103,1984-04-14,Black Brook-West Branch Saint Regis River,14,29,042900060203,0.019425,0.009163,235.181294,32.130492,0.017264,...,13645.402320,4.457617,4246.383527,3.818668,10210.478347,9.604541,763.874938,34.874388,NaN,NaN
60900,1984-04-14,Long Pond Outlet,14,29,042900060202,0.018082,0.008946,246.809985,19.527763,0.061775,...,13653.444991,6.092611,4247.593827,3.930360,10216.761584,11.724158,745.281888,40.428919,NaN,NaN
60697,1984-04-14,Windfall Brook-West Branch Saint Regis River,14,29,042900060201,0.023573,0.010893,240.343958,26.571591,0.051785,...,13661.354888,7.572178,4235.257635,3.416223,10247.124004,14.690527,647.611469,39.980944,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104541,2024-02-08,Bear Creek,14,30,041403010201,0.012929,0.018892,9404.302390,1100.009550,0.013757,...,15394.638115,1.364813,6250.914828,1.391706,10135.666639,0.858703,836.215051,8.757617,0.008237,0.001290
71413,2024-02-08,Hill Creek-East Stony Creek,14,30,020200020502,0.022832,0.026001,9086.706658,1693.267372,0.025057,...,15488.675119,2.611643,6209.140961,1.780507,13165.917240,770.192760,169.093853,18.178793,0.007429,0.002135
6089,2024-02-08,Mayfield Creek-Sacandaga River,14,29,020200020605,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105955,2024-02-08,Balsam Creek-Beaver River,14,30,041403011103,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
selected_columns = ['DATE_SMP', 'WRS_PATH', 'WRS_ROW', 'PONDNAME', 'huc12', 'B6_mean']
selected_df = df_filtered_lakes[selected_columns]
selected_df

,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean
0,1984-04-14,14,29,Upper Kayaderosseras Creek,020200030401,NaN
18676,1984-04-14,14,29,Valentine Pond-Schroon River,020200010605,NaN
61103,1984-04-14,14,29,Black Brook-West Branch Saint Regis River,042900060203,82.435254
60900,1984-04-14,14,29,Long Pond Outlet,042900060202,45.837491
60697,1984-04-14,14,29,Windfall Brook-West Branch Saint Regis River,042900060201,62.751975
...,...,...,...,...,...,...
104541,2024-02-08,14,30,Bear Creek,041403010201,18674.354964
71413,2024-02-08,14,30,Hill Creek-East Stony Creek,020200020502,18156.357989
6089,2024-02-08,14,29,Mayfield Creek-Sacandaga River,020200020605,NaN
105955,2024-02-08,14,30,Balsam Creek-Beaver River,041403011103,NaN


In [6]:
selected_df.dropna(inplace=True)
selected_df

C:\Users\Faculty\AppData\Local\Temp\ipykernel_18468\1280864149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df.dropna(inplace=True)


,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean
61103,1984-04-14,14,29,Black Brook-West Branch Saint Regis River,042900060203,82.435254
60900,1984-04-14,14,29,Long Pond Outlet,042900060202,45.837491
60697,1984-04-14,14,29,Windfall Brook-West Branch Saint Regis River,042900060201,62.751975
60494,1984-04-14,14,29,Stony Brook,042900060204,40.689533
60291,1984-04-14,14,29,Trout Brook,042900060302,46.367013
...,...,...,...,...,...,...
104137,2024-02-08,14,30,Little Black Creek,041403010103,18795.408413
15630,2024-02-08,14,29,Niagara Brook-The Branch,020200010502,18097.912531
71615,2024-02-08,14,30,Bear Creek-East Stony Creek,020200020506,18302.308496
104541,2024-02-08,14,30,Bear Creek,041403010201,18674.354964


In [7]:
import pandas as pd

# Convert DATE_SMP to datetime format
selected_df['DATE_SMP'] = pd.to_datetime(selected_df['DATE_SMP'])

# Filter rows based on the year
start_year = 1984
end_year = 2011
filtered_df_l5 = selected_df[(selected_df['DATE_SMP'].dt.year >= start_year) & (selected_df['DATE_SMP'].dt.year <= end_year)]
filtered_df_l5

C:\Users\Faculty\AppData\Local\Temp\ipykernel_18468\892856125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['DATE_SMP'] = pd.to_datetime(selected_df['DATE_SMP'])


,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean
61103,1984-04-14,14,29,Black Brook-West Branch Saint Regis River,042900060203,82.435254
60900,1984-04-14,14,29,Long Pond Outlet,042900060202,45.837491
60697,1984-04-14,14,29,Windfall Brook-West Branch Saint Regis River,042900060201,62.751975
60494,1984-04-14,14,29,Stony Brook,042900060204,40.689533
60291,1984-04-14,14,29,Trout Brook,042900060302,46.367013
...,...,...,...,...,...,...
78215,2011-09-16,14,30,Sand Pond Brook-The Branch,020200010501,99.431286
69731,2011-09-16,14,30,Kunjamuk River,020200020303,102.482407
71751,2011-09-16,14,30,Second Pond Brook-East Branch Sacandaga River,020200020101,100.431274
113161,2011-09-16,14,30,McKenzie Brook-Lake Champlain,043001080304,112.396236


In [8]:
filtered_df_l5['L6_mean'] = 0.055375 * filtered_df_l5['B6_mean'] + 1.18243
filtered_df_l5

C:\Users\Faculty\AppData\Local\Temp\ipykernel_18468\972580030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_l5['L6_mean'] = 0.055375 * filtered_df_l5['B6_mean'] + 1.18243


,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean,L6_mean
61103,1984-04-14,14,29,Black Brook-West Branch Saint Regis River,042900060203,82.435254,5.747282
60900,1984-04-14,14,29,Long Pond Outlet,042900060202,45.837491,3.720681
60697,1984-04-14,14,29,Windfall Brook-West Branch Saint Regis River,042900060201,62.751975,4.657321
60494,1984-04-14,14,29,Stony Brook,042900060204,40.689533,3.435613
60291,1984-04-14,14,29,Trout Brook,042900060302,46.367013,3.750003
...,...,...,...,...,...,...,...
78215,2011-09-16,14,30,Sand Pond Brook-The Branch,020200010501,99.431286,6.688437
69731,2011-09-16,14,30,Kunjamuk River,020200020303,102.482407,6.857393
71751,2011-09-16,14,30,Second Pond Brook-East Branch Sacandaga River,020200020101,100.431274,6.743812
113161,2011-09-16,14,30,McKenzie Brook-Lake Champlain,043001080304,112.396236,7.406372


In [9]:
import pandas as pd

# Convert DATE_SMP to datetime format
selected_df['DATE_SMP'] = pd.to_datetime(selected_df['DATE_SMP'])

# Filter rows based on the year
start_year = 2013
end_year = 2024
filtered_df_l8 = selected_df[(selected_df['DATE_SMP'].dt.year >= start_year) & (selected_df['DATE_SMP'].dt.year <= end_year)]
filtered_df_l8

C:\Users\Faculty\AppData\Local\Temp\ipykernel_18468\3650758860.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['DATE_SMP'] = pd.to_datetime(selected_df['DATE_SMP'])


,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean
73368,2013-05-16,14,30,Patterson Brook-Hudson River,020200010703,23331.663673
54340,2013-05-16,14,29,Black River,043001080702,25107.492598
67914,2013-05-16,14,30,West Stony Creek,020200020401,25126.439216
57791,2013-05-16,14,29,South Branch Grass River,042900040202,25452.766954
73166,2013-05-16,14,30,Glen Creek,020200010701,24868.854402
...,...,...,...,...,...,...
104137,2024-02-08,14,30,Little Black Creek,041403010103,18795.408413
15630,2024-02-08,14,29,Niagara Brook-The Branch,020200010502,18097.912531
71615,2024-02-08,14,30,Bear Creek-East Stony Creek,020200020506,18302.308496
104541,2024-02-08,14,30,Bear Creek,041403010201,18674.354964


In [10]:
# filtered_df_l8['L6_mean'] = 0.055375 * filtered_df_l8['B6_mean'] + 1.18243
filtered_df_l8['L6_mean'] = 0.0003342 * filtered_df_l8['B6_mean'] + 0.1

# B10*0.0003342+0.1
filtered_df_l8

C:\Users\Faculty\AppData\Local\Temp\ipykernel_18468\272411674.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_l8['L6_mean'] = 0.0003342 * filtered_df_l8['B6_mean'] + 0.1


,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean,L6_mean
73368,2013-05-16,14,30,Patterson Brook-Hudson River,020200010703,23331.663673,7.897442
54340,2013-05-16,14,29,Black River,043001080702,25107.492598,8.490924
67914,2013-05-16,14,30,West Stony Creek,020200020401,25126.439216,8.497256
57791,2013-05-16,14,29,South Branch Grass River,042900040202,25452.766954,8.606315
73166,2013-05-16,14,30,Glen Creek,020200010701,24868.854402,8.411171
...,...,...,...,...,...,...,...
104137,2024-02-08,14,30,Little Black Creek,041403010103,18795.408413,6.381425
15630,2024-02-08,14,29,Niagara Brook-The Branch,020200010502,18097.912531,6.148322
71615,2024-02-08,14,30,Bear Creek-East Stony Creek,020200020506,18302.308496,6.216631
104541,2024-02-08,14,30,Bear Creek,041403010201,18674.354964,6.340969


In [11]:
import pandas as pd

# Assuming filtered_df_l5 and filtered_df_l8 are your DataFrames
concatenated_df = pd.concat([filtered_df_l5, filtered_df_l8], ignore_index=True)
concatenated_df

,DATE_SMP,WRS_PATH,WRS_ROW,PONDNAME,huc12,B6_mean,L6_mean
0,1984-04-14,14,29,Black Brook-West Branch Saint Regis River,042900060203,82.435254,5.747282
1,1984-04-14,14,29,Long Pond Outlet,042900060202,45.837491,3.720681
2,1984-04-14,14,29,Windfall Brook-West Branch Saint Regis River,042900060201,62.751975,4.657321
3,1984-04-14,14,29,Stony Brook,042900060204,40.689533,3.435613
4,1984-04-14,14,29,Trout Brook,042900060302,46.367013,3.750003
...,...,...,...,...,...,...,...
65617,2024-02-08,14,30,Little Black Creek,041403010103,18795.408413,6.381425
65618,2024-02-08,14,29,Niagara Brook-The Branch,020200010502,18097.912531,6.148322
65619,2024-02-08,14,30,Bear Creek-East Stony Creek,020200020506,18302.308496,6.216631
65620,2024-02-08,14,30,Bear Creek,041403010201,18674.354964,6.340969


In [12]:
# Define the file path for the Excel file
excel_file_path = 'L5+L8_WATERSHED_Tier1_AtmCorrected.xlsx'

# Export the DataFrame to Excel
concatenated_df.to_excel(excel_file_path, index=False)

print("DataFrame successfully exported to Excel file:", excel_file_path)


DataFrame successfully exported to Excel file: L5+L8_WATERSHED_Tier1_AtmCorrected.xlsx


In [13]:
import pandas as pd
from scipy import stats
import numpy as np

# Your code to compute significant slopes goes here...

# Create a list to store the results
results = []
concatenated_df['DATE_SMP'] = pd.to_datetime(concatenated_df['DATE_SMP'])

# Loop over each lake in the DataFrame
for lake_id, lake_data in concatenated_df.groupby('huc12'):
    pond_n = concatenated_df.loc[concatenated_df['huc12'] == lake_id, 'PONDNAME'].iloc[0]

    # Loop over each month from April to December
    for month in range(1, 13):
        # Filter data for the specific month and lake
        month_data = lake_data[lake_data['DATE_SMP'].dt.month == month]
        
        # Check if the data is not empty
        if not month_data.empty:
            # Compute the linear regression
            x = month_data['DATE_SMP'].values.astype(np.int64) // (10 ** 9)  # Convert slope to °C/decade # Convert to seconds
            y = month_data['L6_mean'].values
            
            # Check if x and y contain more than one value
            if len(np.unique(x)) > 1:
                slope, _, r_value, p_value, _ = stats.linregress(x, y)
                
                # Check if the trend is significant (p-value < 0.05)
                if p_value < 0.05:
                    slope_per_decade = slope * 10 * 365 * 24 * 3600
                    result = {
                        'Lake_ID': lake_id,
                        'Lake_name': pond_n,
                        'Month': month,
                        'Slope_per_Decade': slope_per_decade,
                        'P-value': p_value,
                        'R-value': r_value
                        	
                    }
                    results.append(result)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('Significant_SLOPES_L5+L8_WATERSHED_Tier1_AtmCorrected.csv', index=False)

print("Results saved to Significant_SLOPES_L5+L8_WATERSHED_Tier1_AtmCorrected.csv")

Results saved to Significant_SLOPES_L5+L8_WATERSHED_Tier1_AtmCorrected.csv


In [16]:
import pandas as pd
from scipy import stats
import numpy as np

# Your code to load and prepare the data...

# Create a list to store the results
results = []

# Convert DATE_SMP to datetime
concatenated_df['DATE_SMP'] = pd.to_datetime(concatenated_df['DATE_SMP'])

# Filter data for the month of August (month number 8)
august_data = concatenated_df[concatenated_df['DATE_SMP'].dt.month == 8]

# Loop over each lake in the filtered August data
for lake_id, lake_data in august_data.groupby('huc12'):
    pond_n = lake_data['PONDNAME'].iloc[0]

    # Compute the linear regression
    x = lake_data['DATE_SMP'].values.astype(np.int64) // (10 ** 9)  # Convert to seconds
    y = lake_data['L6_mean'].values

    # Check if x and y contain more than one value
    if len(np.unique(x)) > 1:
        slope, _, r_value, p_value, _ = stats.linregress(x, y)

        # Check if the trend is significant (p-value < 0.05)
        if p_value < 0.05:
            slope_per_decade = slope * 10 * 365 * 24 * 3600
            result = {
                'Lake_ID': lake_id,
                'Lake_name': pond_n,
                'Month': 8,  # August
                'Slope_per_Decade': slope_per_decade,
                'P-value': p_value,
                'R-value': r_value
            }
            results.append(result)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
results_df.to_csv('Significant_SLOPES_August_WaterShed_Tier1_AtmCorrected.csv', index=False)

print("Results saved to Significant_SLOPES_August_1000Lakes_Tier1_AtmCorrected.csv")


Results saved to Significant_SLOPES_August_1000Lakes_Tier1_AtmCorrected.csv
